# Computing Green FK spectra with GEMINI for use with ASKI

## Introductory remarks

If you want to compute waveform sensitivity kernels using ASKI based on GEMINI solutions of the elastic wave equation, this is the first step you need to take.

To compute waveform sensitivity kernels in ASKI, we need wavefields radiating from seismic sources into the model domain and wavefields radiating from the receiver positions into the model domain.
The kernels are obtained by convolution of the two wavefields which is equivalent to a multiplication in the frequency domain.

As a first step, we configure here the computation of Green functions in the frequency-wavenumber (FK) domain for a given 1D earth model using the program <b>computeGreenFKSpectraForASKI</b>. From these, the required seismic wavefields in the model domain are obtained in a second step using the programs <b>computeKernelWavefield</b> and <b>computeKernelGreenTensor</b>. 

GEMINI uses externally defined radial nodes where solutions are stored and which at the same time serve as potential source positions. <b>computeGreenFKSpectraForASKI</b> considers every radial node as a source node and calculates the resulting Green FK-spectra at all the other nodes. In case the range of sources is smaller than the range covered by the radial nodes, you may set a minimum source radius. For each source node, a separate output file is written to disk. Besides the source type, the source node index and, in case of a parallel run the processor's rank, are appended to the file's base name. But no worry, <b>computeKernelWavefield</b>, <b>computeKernelGreenTensor</b> and <b>plotGreenFKSpectra</b> know about how to read these files correctly.

## Creating a working folder

You should first create a new working folder where you want to run <b>computeGreenFKSpectraForASKI</b> from and to which parameter files and output files are written. Doing so helps in organizing things and avoiding inconsistencies between in- and output. If you haven't done so yet, please create that folder and copy this notebook to the new place. Then, close and halt this notebool, switch to the working folder and start the copy from there.

## Creating the parameter file

In the following, it is assumed that you are now in your new working folder. The first thing we do is to create the parameter file for <b>computeGreenFKSpectraForASKI</b>. To do this we first append the folder where the Python parameter templates of GEMINI are located to the path variable. Please enter the appropriate folder name.

In [1]:
import sys
sys.path.append('/home/wolle/work_git/geminiUnified/python/')

### Earth model

The GEMINI code needs an Earth model in <i>nodeEarthmodel</i> format. Here, an example how it looks.
1.  keyword saying it is in <i>nodeEarthmodel</i> format
2.  arbitrary text
3.  reference frequency in Hz, attenuation flag (not used), anisotropy flag, earth radius in km
4.  number of lines following
5.  depth in meters, density in g/cm$^3$, P-wave velocity in km/s, S-wave velocity in km/s, quality factor for $\kappa$, $Q_{\kappa}$, quality factor for $\mu$, $Q_\mu$.

Note:
+  first line gives material properties at the surface
+  discontinuities are specified by double nodes
+  single nodes act as anchor points for a <b>continuous</b> depth variation between two discontinuities. <b>computeGreenFKSpectraForASKI</b> uses spline interpolation for this purpose.

In [2]:
earth_model = 'ak135q.nm'

### External radial nodes

Here you may define the set of radial nodes GEMINI uses as potential source nodes and as places where the Green FK spectra are stored. The length unit is <b>km</b>. Make sure that all radial nodes are above the inner homogeneous sphere of the earth model. Note: Indexing of radial nodes increases with radius not depth. Thus, index 1 corresponds to the bottommost node.

First we define the radius of the earth in km. You may choose a different value, for example when computing wave propagation in a tennis ball.

In [3]:
rearth = 6371.

The nodes are organized into several blocks within which nodes are equidistant.

In [4]:
nblocks = 2

In contrast to the ordering of the nodes, the blocks are ordered from top to bottom. The following integer array specifies the number of nodes within each block:

In [5]:
nnod = '10 10'

The following real array specifies the spacing between the nodes of each block, also ordered from top to bottom. Unit is km.

In [6]:
dr = '3 5'

By default, there is always one node at the surface. However, you may shift this node and with it all other nodes below to greater depths:

In [7]:
shift = 0.0

### Miscellaneous parameters

Maximum length of synthetics to be calculated from Green FK-spectra in seconds.
Determines the frequency stepping: df = 1/tlen:

In [8]:
tlen = 512

Distance from which wavenumber stepping is calculated in km
(should be about 10 times the maximum source receiver distance):

In [9]:
xlen = 5000.

Upper frequency limit of Green FK-spectra in Hz 
(should be twice the Nyquist frequency desired for synthetics):

In [10]:
fmax = 0.2

Basename of output files for Green FK-spectra. Program appends <i>'.'+sourcetype+'.'+source_node+'.'+rank</i>, where <i>sourcetype</i> is either FORCE or MOMENT, <i>source_node</i> is the index of the source radial node and <i>rank</i> is the rank of the processor by which the file was written:

In [11]:
basename = 'gfk'

Source type: set 'FORCE' for single force or 'MOMENT' for moment tensor source:

In [12]:
source_type = 'MOMENT'

Mode of attenuation. This replaces the attenuation flag in the earth model file which was used in older versions. Allowed values for this parameter are:<br>
ELASTIC, ATTENUATION_ONLY, DISPERSION_ONLY, ATTENUATION_AND_DISPERSION

In [13]:
attn_mode = 'ATTENUATION_AND_DISPERSION'

Accuracy of computation. Reasonable values are $10^{-4}$ to $10^{-6}$. The adaptive step size integration method stops refining step size once the estimated error is below this value.

In [14]:
eps = 1.e-5

We also need an upper limit for the wavenumbers. Since this depends linearly on frequency, we instead give a slowness or inverse wave speed limit $p=k/\omega$ expressed in units of s/km. You may specifiy two different values to save computation time. The first one is for the frequency range $0 < f < f_{max}/2$ and the second one for the range $f_{max}/2 < f < f_{max}$. The 2nd one can be more restrictive because contributions by frequencies beyond $f_{max}/2$ will be diminished by a low-pass filter when tranforming into the time domain. Given a value of $p$, waves travelling with apparent speeds of $1/p$ will miss in the synthetic seismograms. A good first estimate for $p$ is the slowness of the slowest surface wave propagating in the model.

In [15]:
slowness_limit_1 = 0.5
slowness_limit_2 = 0.5

Since a formula $k_{max} = p\omega$ would lead to $k_{max} = 0$ at zero frequency, the wavenumber range is extended by a constant fraction of the one at $f_{max}/2$. Here, you can specify this fraction:

In [16]:
wavenumber_margin_fraction = 0.1

The range of nodes onsidered as source nodes can be restricted by defining a lower node limit (remember nodes are numbered bottom up):

In [17]:
min_source_node = 1

Finally, we need to specify the radial node where the receivers are located (typically at the surface):

In [18]:
rec_node_index = 21

Now we are ready to create the parameter file. First, we generate a Python string containing the entire contents fo the file and then we write it to the working folder:

In [19]:
from parfile_gfkaski import create_gfkaski_parfile 

parstring = create_gfkaski_parfile(tlen,xlen,fmax,earth_model,basename,
            source_type,attn_mode,eps,slowness_limit_1,slowness_limit_2,wavenumber_margin_fraction,
            rearth,nblocks,nnod,dr,shift,min_source_node,rec_node_index)
outfile = './'+'gfkaski_parfile'

In [87]:
with open(outfile,'w') as fd:
    fd.write(parstring)